In [ ]:
#SEE IF graphs look better with or without "%matplotlib inline" for you;
%matplotlib inline
import os

import numpy as np
import matplotlib.pyplot as plt

from scipy.ndimage.filters import gaussian_filter
#from matplotlib.pyplot import contour, show

from collections import namedtuple
from mpl_toolkits.mplot3d import Axes3D
import random

bench_dir = 'bench_py_v010'

size_ag, size_lex, size_mem = 32, 32, 32

#varexam='AcceptanceRate'
varexam='Comprehension'
#varexam='GrammarVar'


mutators = ['ProbitVectorGaussian', 'ProbitVectorUniform',\
            'AdditiveVectorUniform', 'ProbitSingleGaussian',\
            'ProbitSingleUniform', 'AdditiveSingleUniform', 'NoMutation',\
            'SingleBeta', 'VectorDirichlet', 'SingleGamma', 'VectorLogMultivariateGamma',\
            'AdditiveSingleClipExppGaussian', 'AdditiveVectorClipExppGaussian',\
            'AdditiveSingleAbsExppGaussian', 'AdditiveVectorAbsExppGaussian',\
            'AdditiveSingleExppUniform', 'AdditiveVectorExppUniform',\
            'ArcsSingleClip', 'ArcsVectorClip']

#mrspan_start, mrspan_stop, mrspan_points = 0.0, 2.0, 20+1
mrspan_start, mrspan_stop, mrspan_points = 0.2, 1.0, 4+1

mr_seq = np.linspace(mrspan_start, mrspan_stop, num=mrspan_points, endpoint = True)
#print("Sampled values: %s", mr_seq)
plt.rcParams['figure.figsize'] = (15,15)
plt.rcParams['figure.dpi'] = 150

def dumpread(fn):
    rawreadgen = np.genfromtxt(fn, names = True, delimiter=' ', skip_header = 2, invalid_raise = False, encoding = None)
    cols = rawreadgen.dtype.names
    #Sprint("Read file %s, %i observations." % (fn, len(rawreadgen)))
    #print("...read <%s> columns." % ', '.join(cols) )
    return rawreadgen

acc_rate_matrix = dict()

for mutator in mutators:
    acc_rate_matrix[mutator] = dict()
    for mr_val in mr_seq:
        #sorry, varying number of decimal values makes for crappy code.
        fname = 'run_%d_%d_%d_mr_{%.2f}_{%s}.txt' % (size_ag, size_lex, size_mem, mr_val, mutator)
        ffname = os.path.join(bench_dir, fname)
        dg = dumpread(ffname)
        #acc_rate_matrix['%.1f' % mr_val] = dg
        acc_rate_matrix[mutator]['%.2f' % mr_val] = dg

    print("Read %s dump files." % mutator)

In [ ]:
#PLOT TIME SERIES
#PLOT Mutation min, average, max across runs values, by mutator (can also look at percentiles, just in case)

obsnum = 100

#percentiles = [5, 95]
percentiles = [0, 100] #min/max

multirun_avg = dict()
multirun_low = dict()
multirun_high = dict()

for mutator in mutators:
    multirun_avg[mutator] = list()
    multirun_low[mutator] = list()
    multirun_high[mutator] = list()
    for obs in range(0, obsnum):
        vals = list( acc_rate_matrix[mutator][k][varexam][obs] for k in acc_rate_matrix[mutator].keys() )
        avg = np.average( vals )
        low = np.percentile( vals, percentiles[0] )
        high = np.percentile( vals, percentiles[1] )
        multirun_avg[mutator].append(avg)
        multirun_low[mutator].append(low)
        multirun_high[mutator].append(high)


In [ ]:
#subplots columns, 4 seems the most readable
spCols = 4

f, unfl_axarr = plt.subplots(len(mutators)/spCols+1, spCols)#, sharey=True, sharex=True)
axarr=unfl_axarr.flatten()

x = range(0, obsnum)
for axindex, mutator in enumerate(mutators, start = 0):
    #print mutator,multirun_avg[mutator]
    y = multirun_avg[mutator][0:obsnum]
    y_low = multirun_low[mutator][0:obsnum]
    y_high = multirun_high[mutator][0:obsnum]
    #axarr[axindex].set_title("%s | %s" % (mutator, varexam))
    axarr[axindex].set_title("%s" % mutator)

    axarr[axindex].plot(x,y_high, label = 'high')
    axarr[axindex].plot(x,y, label = 'mean')
    axarr[axindex].plot(x,y_low, label = 'low')
    
    axarr[axindex].fill_between(x, y_low, y_high, alpha = .2, color = 'grey' )
    if (axindex == 0):
        axarr[axindex].legend(loc = 'upper right')
    axindex += 1
    print("Min/Max %s values for mutator %s are %.3f, %.3f" % (varexam, mutator, min(y), max(y)) )

plt.tight_layout()
plt.show()


In [ ]:
for mutator in mutators:#[0:1]:

    fig = plt.figure()

    ax = fig.add_subplot(111, projection='3d')

    x = list( map(float, sorted(acc_rate_matrix[mutator].keys())) )
    y = range(0, obsnum)

    X, Y = np.meshgrid(x, y)

    zs = np.array( [acc_rate_matrix[mutator]['%0.2f' % (j)][varexam][k] for k in y for j in x] )
    Z = zs.reshape(X.shape)

    #sigma = 0.8 # this depends on how noisy your data is, play with it!
    #Z = gaussian_filter(Z, sigma)

    #ax.plot_surface(X, Y, Z, cmap = 'Greys', alpha = 0.8) #USE GRAYSCALE
    ax.plot_surface(X, Y, Z, cmap = 'RdBu_r', alpha = 0.8)

    ax.set_title(mutator)
    ax.set_xlabel(mutator+' Mutation Scale')
    ax.set_ylabel('Iteration')
    ax.set_zlabel(varexam)

    plt.show()